# Two Tower Passage Ranking applyed to MS MARCO
### Desenvolvido por Graziella Bonadia e Matheus Sasso

## Modelo Escolhido

In [ ]:
#@title Configurações gerais
experiment_name = 'TwoTowerPassageRanking'  #@param {type:"string"}
model_name = 'bert-base-uncased'  #@param ["bert-base-uncased","bert-large-uncased","albert-base-v2"] {type:"string"}


## Instalações Externas e Download dos dados

Instalação de pacotes

In [ ]:
! pip install pytorch-lightning  --quiet
! pip install transformers  --quiet
! pip install ftfy --quiet

     |████████████████████████████████| 296kB 13.3MB/s 
     |████████████████████████████████| 276kB 33.5MB/s 
     |████████████████████████████████| 829kB 45.3MB/s 
     |████████████████████████████████| 675kB 12.5MB/s 
     |████████████████████████████████| 1.1MB 56.2MB/s 
     |████████████████████████████████| 3.8MB 56.9MB/s 
     |████████████████████████████████| 890kB 51.0MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 


In [ ]:
%%capture
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl
#!apt install libomp-dev
#!python -m pip install --upgrade faiss faiss-gpu

In [ ]:
import faiss

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


Recuperação de **funções auxiliares**

In [ ]:
%%capture
!wget -nc https://raw.githubusercontent.com/Matheus158257/ms-marco-passage-ranking-dense-vectors-with-doc2query/master/read_ms_marco.py
!wget -nc https://raw.githubusercontent.com/Matheus158257/ms-marco-passage-ranking-dense-vectors-with-doc2query/master/read_ms_marco2.py

Importando google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !gzip -d "/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos/qidpidnid/qidpidtriples.train.full.2.tsv.gz"
# import shutil
# !tar xvf  "/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos/qidpidnid/qidpidtriples.train.full.2.tsv.gz" -C "/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos/qidpidnid/"
# !unzip "/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos/t5_base/t5-base.zip" -d "/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos/t5_base"

Variáveis globais dos caminhos para os arquivos importandos

In [ ]:
import os
# proj_dir = '/content/drive/My Drive/Mestrado/PLN/Projeto'#Matheus
# data_base_dir = '/content/drive/My Drive/Mestrado/PLN/Projeto/Data/Vetores_Densos' #Matheus
proj_dir = '/content/drive/My Drive/Projeto'#Matheus
data_base_dir = '/content/drive/My Drive/Projeto/Data/Vetores_Densos' #Matheus
# proj_dir = '/content/drive/My Drive/Projeto/Colabs'#Graziella
# data_base_dir = '/content/drive/My Drive/Projeto/Data/Vetores_Densos' #Graziella

Comum para os dois

In [ ]:
t5_zip_path = data_base_dir + '/t5_base/model.ckpt-1004000'
qrels_train_path = data_base_dir + '/collectionandqueries/qrels.train.tsv'
qrels_dev_small_path = data_base_dir + '/collectionandqueries/qrels.dev.small.tsv'
collection_path = data_base_dir + '/collectionandqueries/collection.tsv'
queries_train_path = data_base_dir + '/collectionandqueries/queries.train.tsv'
queries_eval_path = data_base_dir + '/collectionandqueries/queries.eval.tsv'
queries_eval_small_path = data_base_dir + '/collectionandqueries/queries.eval.small.tsv'
queries_dev_path = data_base_dir + '/collectionandqueries/queries.dev.tsv'
queries_dev_small_path = data_base_dir + '/collectionandqueries/queries_dev_small.tsv'
# queries_topk_folder  = data_base_dir + '/predicted_queries_topk_sampling'
queries_topk_path  = data_base_dir + '/top1_K/doc2query_onek.tsv'
# train_triples_small_path = data_base_dir + '/train_triples_small/triples_smalls_chunk_1.csv'
qid_pid_nid_path = data_base_dir + '/qidpidnid/triples_ids_train.tsv'
inference_data100_path = data_base_dir +'/inference_data100.json'
inference_data100_eval_2020_path = data_base_dir + '/inference_data100_eval_2020.json'
ten_negative_triples_path = data_base_dir + '/ten_negatives_triples.json'

Para a utilização de TPU

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

## Imports

In [ ]:
# Importar todos os pacotes de uma só vez para evitar duplicados ao longo do notebook.

import nvidia_smi
import math
from torch.nn.functional import softmax
from google.colab import drive
from pytorch_lightning.callbacks import ModelCheckpoint

import pytorch_lightning as pl

from torch.autograd import Variable
from sklearn.metrics import accuracy_score
import faiss

from transformers import T5ForConditionalGeneration
from transformers import T5Tokenizer

# General Libs
import gzip
import random
import pdb
import ftfy

#Hardware Data
from multiprocessing import cpu_count
import psutil

#import das funções que vem do github
import collections
import functools
import traceback
import sys
import os
from read_ms_marco2 import load_qrels
from read_ms_marco2 import load_queries
from read_ms_marco import load_collection
from read_ms_marco import load_doc2query
from read_ms_marco import load_triple
from read_ms_marco2 import load_txts_topk #load_txts_topk(folder,k=1,n=18,encoding="cp1252")

#Data Sciens Libs
import numpy as np
import pandas as pd

#Lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

#Torch
import torch
from torch import Tensor
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn.functional as F

# TORCH XLA - TPU
# import torch_xla.core.xla_model as xm

#Transformers
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained(model_name)

#Generic Types
from typing import Dict
from typing import List
from typing import Tuple


## Configurações Gerais

Imperdir Excesso de Logs

In [ ]:
# import logging
# logging.getLogger("transformers.configuration_utils").setLevel(logging.WARNING)
# logging.getLogger("transformers.modeling_utils").setLevel(logging.WARNING)
# logging.getLogger("lightning").setLevel(logging.WARNING)

Dados de CPU e GPU

In [ ]:
from multiprocessing import cpu_count
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
print("\nGetting Hardware Statatus...\n")
def hardware_stats():
    '''
    Returns a dict containing some hardware related stats
    '''
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    return {"cpu": str(psutil.cpu_percent()) + '%',
            "mem": str(psutil.virtual_memory().percent) + '%',
            "gpu": str(res.gpu) + '%',
            "gpu_mem": str(res.memory) + '%'}

print(f"Imports loaded succesfully. Current GPU: {torch.cuda.get_device_name(0)}, number of CPU cores: {cpu_count()}")


Getting Hardware Statatus...

Imports loaded succesfully. Current GPU: Tesla P4, number of CPU cores: 2


Decorator para impedir quebra de memório recorrente

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#Impedir quebra de memória 
# https://docs.fast.ai/troubleshoot.html#memory-leakage-on-exception
import functools, traceback
def gpu_mem_restore(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper

## Preparando Dados

Download do dataset de dev 

In [ ]:
# !wget https://git.uwaterloo.ca/jimmylin/anserini-indexes/raw/master/index-msmarco-passage-20191117-0ed488.tar.gz
# !tar xvfz index-msmarco-passage-20191117-0ed488.tar.gz

Extracao das partes do dataset em triplas

In [ ]:
original_queries_train = load_queries(queries_train_path)
original_queries_dev = load_queries(queries_dev_path)
original_queries_eval = load_queries(queries_eval_path)

FileNotFoundError: ignored

In [ ]:
artificial_queries = load_doc2query(queries_topk_path)

In [ ]:
artificial_queries['842095']

In [ ]:
artificial_queries['842095']

In [ ]:
# from read_ms_marco2 import load_collection
# collections = load_collection(collection_path)

In [ ]:
triples = load_triple(qid_pid_nid_path)

In [ ]:
def take_part(d,n_elements):
  elements =  [str(x) for x in range(n_elements + 1)]
  newdict = {key:d[key] for key in elements}
  return newdict

In [ ]:
def reset_dict_keys(original_dict): 	
   new_dict = {} 	
   for i, (k, v) in enumerate(original_dict.items()):
      new_dict[str(i)] = v
   return new_dict

In [ ]:
def train_val_test(d, tr, vl, ts):
  elements =  [str(x) for x in range(tr)]
  train = {key:d[key] for key in elements}
  elements =  [str((x + tr)) for x in range(vl)]
  val = {key:d[key] for key in elements}
  val = reset_dict_keys(val)
  elements =  [str((x + tr + vl)) for x in range(ts)]
  test = {key:d[key] for key in elements}
  test = reset_dict_keys(test)
  return train, val, test

In [ ]:
debug_train, debug_val, debug_test = train_val_test(triples, 10, 5, 3)

In [ ]:
print(debug_train)
print(debug_val)
print(debug_test)

## Carregando o dataset

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
tokenizer.encode_plus('we like pizza',
                      max_length =10,
                      pad_to_max_length=True,
                      add_special_tokens = True)

Dataset Treino


In [ ]:
class TrainDataset(Dataset):
    def __init__(self, triples_ten_negatives, queries, artificial_queries,
                 tokenizer,max_length: int =100,training_step=False):

        self.triples_ten_negatives = triples_ten_negatives
        self.queries = queries
        self.artificial_queries = artificial_queries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.triples_ten_negatives)

    def __getitem__(self, idx):
      # Numero(Chave)
        # import pdb;pdb.set_trace()
 
        qpdis, nids = self.triples_ten_negatives[str(idx)]
        qid = qpdis.split("|")[0]
        pid = qpdis.split("|")[1]
        
        
        #vetores densos - Texto(Valor)
        o_query = ftfy.fix_text(self.queries[qid])
        a_query_pos = ftfy.fix_text(self.artificial_queries[pid])
        a_queries_neg = [ftfy.fix_text(self.artificial_queries[nid]) for nid in nids] 

        #Tensores
        q_tok, q_mask = self.encode_plus(o_query)
        p_tok, p_mask = self.encode_plus(a_query_pos)
        negative_passage_tensors = [self.encode_plus(a_query_neg) for a_query_neg in a_queries_neg]

        return  (qid, pid, nids, q_tok, q_mask, p_tok, p_mask, negative_passage_tensors)

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        # tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)

        return tok,mask

Dataset Inferência

In [ ]:
class InferenceDataset(Dataset):

    def __init__(self,dicts, queries,artificial_queries,
                 tokenizer,max_length: int =100,training_step=False):
        self.dicts = dicts
        self.queries = queries
        self.passages = artificial_queries
        self.tokenizer = tokenizer
        self.max_length = max_length


    def __len__(self):
      return len(self.dicts)


    def __getitem__(self, idx):

        qid, pids = self.dicts[str(idx)]
        o_query = ftfy.fix_text(self.queries[qid])
        passages = [ftfy.fix_text(self.passages[pid]) for pid in pids]

        o_query_tok,o_query_mask = self.encode_plus(o_query) 
        passages_tensors = [self.encode_plus(passage) for passage in passages] #retorna tuplas com token e mask [(tok,mask),(),(),()] , tok([[tensor()]][tensor()][tensor()][tensor()])

        
        return  (qid,pids,o_query,passages,o_query_tok,o_query_mask,passages_tensors)

    def encode_plus(self, text):
        tokens = self.tokenizer.encode_plus(text=text, max_length=self.max_length,
                                       pad_to_max_length=True, add_special_tokens = True)
        
        
        tok =  torch.tensor(tokens["input_ids"]).type(torch.long)
        mask = torch.tensor(tokens['attention_mask']).type(torch.long)
        # tok_type = torch.tensor(tokens['token_type_ids']).type(torch.long)

        return tok,mask

In [ ]:
import json
with open(inference_data100_eval_2020_path) as json_file:
    inference_test_dict_2020 = json.load(json_file)

In [ ]:
with open(ten_negative_triples_path) as json_file:
    ten_negative_triples_dict= json.load(json_file)

## Testando o DataLoader

In [ ]:
dataset_debug = TrainDataset(
    triples_ten_negatives =  ten_negative_triples_dict,
    queries = original_queries_train,
    artificial_queries = artificial_queries,
    tokenizer=tokenizer,
    max_length=20)

dataloader_debug = DataLoader(dataset_debug, batch_size=2, shuffle=True,num_workers=cpu_count())

qid, pid, nids, q_tok, q_mask, p_tok, p_mask, negative_passage_tensors) = next(iter(dataloader_debug))

        
print('qids:\n', qid)
print('pid:\n', pid)
print('nids:\n', nids)
print('q_tok:\n', q_tok)
print('q_mask:\n', q_mask)
print('p_tok:\n', p_tok)
print('p_mask:\n', p_mask)
print('negative_passage_tensors:\n', negative_passage_tensors)


In [ ]:
 dataset = InferenceDataset(
    dicts = inference_test_dict_2020,
    queries = original_queries_eval,
    artificial_queries = artificial_queries,
    tokenizer=tokenizer,
    max_length=100
)

dataloader_debug = DataLoader(dataset, batch_size=4, shuffle=False,num_workers=cpu_count())
qid,pids,o_query,passages,o_query_tok,o_query_mask,passages_tensors = next(iter(dataloader_debug))# passages_tensors -> lista de tuplas [(pd_tok1,pd_mask1),(pd_tok2,pd_mask2)...]

## Loss Funtion

https://github.com/facebookresearch/faiss/wiki/Getting-started

Faiss Para Negativos como Positivos dos outros valores do Batch

In [ ]:
def faiss_loss(Query,Passage):

     X = Query.cpu()
     Y = Passage.cpu()
     X = X.detach().numpy()
     Y = Y.detach().numpy()

     dd = np.size(X,1)      ##Acertar para torch                    # vector dimension
     batch_size = np.size(X,0)      ##Acertar para torch           # we want to see number of batch size nearest neighbors  
     k = batch_size                                                # Its possible to change this number, but for the sake of this project, it will remain the same number of the batch size
     index = faiss.IndexFlatL2(dd)                                  # build the index
     index.add(Y)                                                  # add vectors to the index

     D, I = index.search(X, k)                                     # search. D is the matriz of similarities and I is the index of the vector in matriz ordered by similarity
    
     #fixed position positive
     T = I==0
     F = I!=0                                           

     D_exp = np.exp(D)                                             # exp of similarities matrix
     sim_pos = D_exp[T==True]                                      # retrieve the vector of similarities of the target, despite its order (counting as positive)
     sim_neg = np.sum(((F == True) * D_exp), axis = 1)             # retrieve the vector of similarities of all other vectors and sum (counting as negative)

     loss = - np.log(sim_pos/(sim_pos+sim_neg))                    # loss
     loss = torch.from_numpy(loss).float().to(device)
     loss = torch.sum(loss)
     loss = Variable(loss, requires_grad=True)

     return loss

Faiss para ranqueamento

In [ ]:
def get_similarity(Query,Passage,k=1):
     X = Query.cpu()
     Y = Passage.cpu()
     X = X.detach().numpy()
     Y = Y.detach().numpy()

     d = np.size(X,1)      ##Acertar para torch                    # vector dimension
     index = faiss.IndexFlatL2(d)                                  # build the index
     index.add(Y)                                                  # add vectors to the index

     D, I = index.search(X, k) 
     return D[0][0]                                    # search. D is the matriz of similarities and I is the index of the vector in matriz ordered by similarity

In [ ]:
def get_similarity2(Query,Passages):
     X = Query.cpu()
     Y = Passages.cpu()
     X = X.detach().numpy()
     Y = Y.detach().numpy()
    
     k = len(Y)
     dim = np.size(X,1)           ##Acertar para torch                   # vector dimension
     index = faiss.IndexFlatL2(dim)                                  # build the index
     index.add(Y)                                                  # add vectors to the index

     D, I = index.search(X, k) 


     return D[0], I[0]

## Hyperparâmetros

In [ ]:
#@title Hyperparâmetros
batch_size =  16#@param {type:"integer"}
max_epochs = 3 #@param {type:"integer"}
accumulate_grad_batches = 16  #@param {type:"integer"}
max_length =   128#@param {type:"integer"}
learning_rate = 5e-3  #@param {type:"number"}

In [ ]:
hyperparams = {'model':model_name,'tokenizer':tokenizer,'learning_rate':learning_rate,'batch_size':batch_size,'max_length':max_length,'accumulate_grad_batches':accumulate_grad_batches} 

## Datasets

In [ ]:
triples_train, _, _ = train_val_test(triples, 50000, 100, 100)

In [ ]:
dataset_train = TrainDataset(triples =  triples_train, queries = original_queries_train,
                          artificial_queries = artificial_queries,
                          tokenizer=tokenizer,
                          max_length=max_length)

dataset_val  = InferenceDataset(
              dicts = inference_test_dict_2020,
              queries = original_queries_eval,
              artificial_queries = artificial_queries,
              tokenizer=tokenizer,
              max_length=max_length)
                              

dataset_test = InferenceDataset(
              dicts = inference_test_dict_2020,
              queries = original_queries_eval,
              artificial_queries = artificial_queries,
              tokenizer=tokenizer,
              max_length=max_length)

train_dataloader = DataLoader(dataset_train, batch_size=batch_size,
                              shuffle=True, num_workers=4)

val_dataloader = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, 
                            num_workers=4)

test_dataloader = DataLoader(dataset_test, batch_size=batch_size,
                             shuffle=False, num_workers=4)

## Criando o Two Tower Passage Ranking com Pytorch Lightning

In [ ]:
class TwoTowerFineTuner(pl.LightningModule):

    def __init__(self, 
                 hyperparams,
                 train_dataloader, val_dataloader,
                 test_dataloader,
                 overfit=False):
      
        super(TwoTowerFineTuner, self).__init__()
        
        
        #---------- Definição do modelo

            ##------ Teste 1
        #self.model = BertModel.from_pretrained(model_name)
            ##------ Teste 2
        self.tower1 = BertModel.from_pretrained(model_name)
        self.tower2 = BertModel.from_pretrained(model_name)
        

        #---------- Hyperparametros
        self.tokenizer = hyperparams['tokenizer']
        self.learning_rate = hyperparams['learning_rate']
        self.batch_size = hyperparams['batch_size']
        self.max_length = hyperparams['max_length']
        self.overfit = overfit

        #---------- Camadas
        self.num_final_dim = 128
        self.dropout = torch.nn.Dropout(self.tower1.config.hidden_dropout_prob)
        self.final_layer = torch.nn.Linear(self.tower1.config.hidden_size, self.num_final_dim)


        #---------- Carregamento datasets (Para eu poder variar self.max_length)

        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader
        self._test_dataloader = test_dataloader
        
        #---------- TXT saving path
        self.results_path = proj_dir + '/Data/Checkpoints_dense_DOIS_MOD/' + 'results.test.txt' 
            

        #---------- Algorythim step
        #self.training = True

    

    def infer(self,batch):

        qid,pids,o_query,passages,o_query_tok,o_query_mask,passages_tensors = batch


        batch_similarities = []
        batch_pids = []
        batch_qids = []
        MRR_batch =  np.array([])
        for i in range(0,len(qid)):
          o_query_toks_tensor = o_query_tok[i].view(1,-1) #1xL 
          o_query_masks_tensor = o_query_mask[i].view(1,-1) #1 X L 
          query_hat = self.forward(o_query_toks_tensor, o_query_masks_tensor,self.tower1)
          phs = []
          for tuple_tensor in passages_tensors: 
            passage_tok = tuple_tensor[0][i].view(1,-1) #1XL 
            passage_mask = tuple_tensor[1][i].view(1,-1)
            #passage_hat = self.forward(passage_tok, passage_mask, self.tower2)
            passage_hat = self.forward(passage_tok, passage_mask, self.tower1)
            phs.append(passage_hat[0])
          
          passage_hats = torch.stack([ph for ph in phs], dim=0)
          similarities, order = get_similarity2(query_hat,passage_hats)#q = 1xL , p = 100 x L
          # similarity_normalized = similarities/np.linalg.norm(similarities)   
          # similarity_normalized = similarities/similarities.max(axis=0) 
          # similarity_normalized_list = [np.float64(elem).item() for elem in similarity_normalized]
          similarity_normalized = [np.float64(elem).item() for elem in similarities][::-1]#revrese
          order_list = [np.float64(elem).item() for elem in order]
          pids_phrase = [p[i] for p in pids]
          pids_ordered = [x for _,x in sorted(zip(order_list,pids_phrase))][::-1] #revrese
          qids = [qid[i]]*len(pids)
          
          MRR = np.sum(similarity_normalized)/len(pids)
          MRR_batch = np.append(MRR_batch,MRR)

          batch_pids.append(pids_ordered)
          batch_qids.append(qids)
          batch_similarities.append(similarity_normalized)

        mrr = torch.tensor(np.mean(MRR_batch))
        return mrr, batch_pids, batch_qids , batch_similarities
      
      
    def forward(self, input_ids, attention_mask, model):    #model

        _, pooled_output = model(input_ids, attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.final_layer(pooled_output)

        return logits
    

    def training_step(self, batch, batch_nb):
        # batch
        qid, pid, nids, q_tok, q_mask, p_tok, p_mask, negative_passage_tensors = batch

        # organize passage data
        loss_list = torch.empty(1) #garbage tensor
        for qt,qm,pt,pm,np_tensor_10 in zip(q_tok,q_mask,p_tok,p_mask,negative_passage_tensors):
          passages_tok_list = [np_tensor[0] for np_tensor in np_tensor_10]
          passages_tok_list.append(pt)
          passages_tok_list = passages_tok_list[::-1]
          passages_toks = torch.stack([passages_tok for passages_tok in passages_tok_list],dim=0)
          passages_mask_list = [np_tensor[1] for np_tensor in np_tensor_10]
          passages_mask_list.append(pm)
          passages_mask_list = passages_mask_list[::-1]
          passages_masks = torch.stack([passages_mask for passages_mask in passages_mask_list],dim=0)


          
          query_hat = self.forward(qt, qm,self.tower1)
          passage_hat = self.forward(passages_tok, passages_mask,self.tower2)

          loss = faiss_loss(query_hat,passage_hat)
          loss_list = torch.cat([loss_list,loss],dim=0)
          
          n_toks_tensor
          n_tok_list.append(np_tensor[0])
          n_mask_list.append(np_tensor[1])

        final_loss  = torch.mean(loss_list[1:])# the first tensor is garbage

        
        # logs
        tensorboard_logs = {'train_loss': final_loss}
        return {'loss': final_loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb): 
        val_mrr, batch_pids, batch_qids , batch_similarities = self.infer(batch)
        return {'val_mrr': val_mrr}
        
    def validation_epoch_end(self, outputs):
        # avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_val_mrr = torch.stack([x['val_mrr'] for x in outputs]).mean()
        tensorboard_logs = {'avg_val_mrr': avg_val_mrr}    
        return {'avg_val_mrr': avg_val_mrr,'progress_bar': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        
        test_mrr, batch_pids, batch_qids , batch_similarities = self.infer(batch)
        
        return {'test_mrr': test_mrr}

    def test_epoch_end(self, outputs):

        avg_test_mrr = torch.stack([x['test_mrr'] for x in outputs]).mean()

        tensorboard_logs = {'avg_test_mrr': avg_test_mrr}
        return {'avg_test_mrr': avg_test_mrr, 'log': tensorboard_logs,
                'progress_bar': tensorboard_logs}
    
    def configure_optimizers(self):
        return torch.optim.Adam(
            [p for p in self.parameters() if p.requires_grad],
            lr=self.learning_rate, eps=1e-08)

    def train_dataloader(self):
        #pdb.set_trace()
        return self._train_dataloader

    def val_dataloader(self):
        #pdb.set_trace()
        return self._val_dataloader

    def test_dataloader(self):
        #pdb.set_trace()
        return self._test_dataloader

In [ ]:
# model = TwoTowerFineTuner(hyperparams, train_dataloader=train_dataloader,
#                       val_dataloader=val_dataloader,
#                       test_dataloader=test_dataloader)

In [ ]:
# sum([torch.tensor(x.size()).prod() for x in model.parameters() if x.requires_grad]) # trainable parameters

tensor(219062912)

## Etapas de avaliação do Lightning

### Testando rapidamente o modelo em treino, validação e teste com um batch

In [ ]:
# trainer = pl.Trainer(gpus=1, 
#                      checkpoint_callback=False,  # Disable checkpoint saving.
#                      fast_dev_run=True)
# trainer.fit(model)
# trainer.test(model)
# del model  # Para não ter estouro de mémoria da GPU

Running in fast_dev_run mode: will run a full train, val and test loop using a single batch
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type      | Params
------------------------------------------
0 | tower1      | BertModel | 109 M 
1 | tower2      | BertModel | 109 M 
2 | dropout     | Dropout   | 0     
3 | final_layer | Linear    | 98 K  


--------------------------------------------------------------------------------
TEST RESULTS
{'avg_test_mrr': tensor(0.2175, dtype=torch.float64)}
--------------------------------------------------------------------------------



In [ ]:
# del model  # Para não ter estouro de mémoria da GPU


### Trabalhando no datasset todo



In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_path = proj_dir + '/Data/Checkpoints_dense_DOIS_MOD/epoch=3.ckpt'
 
checkpoint_dir = os.path.dirname(os.path.abspath(checkpoint_path))
print(f'Files in {checkpoint_dir}: {os.listdir(checkpoint_dir)}')
print(f'Saving checkpoints to {checkpoint_dir}')
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_dir,save_top_k=-1)  # Keeps all checkpoints.

resume_from_checkpoint = None
if os.path.exists(checkpoint_path):
    print(f'Restoring checkpoint: {checkpoint_path}')
    resume_from_checkpoint = checkpoint_path


trainer = pl.Trainer(gpus=1,
                     max_epochs=max_epochs,
                     check_val_every_n_epoch=1,
                     accumulate_grad_batches= accumulate_grad_batches,
                     checkpoint_callback=checkpoint_callback,
                     resume_from_checkpoint=resume_from_checkpoint)


model = TwoTowerFineTuner(hyperparams, train_dataloader=train_dataloader,
                      val_dataloader=val_dataloader,
                      test_dataloader=test_dataloader)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Files in /content/drive/My Drive/Projeto/Data/Checkpoints_dense_DOIS_MOD: ['epoch=1.ckpt', 'epoch=2.ckpt', 'epoch=3.ckpt']
Saving checkpoints to /content/drive/My Drive/Projeto/Data/Checkpoints_dense_DOIS_MOD
Restoring checkpoint: /content/drive/My Drive/Projeto/Data/Checkpoints_dense_DOIS_MOD/epoch=3.ckpt



  | Name        | Type      | Params
------------------------------------------
0 | tower1      | BertModel | 109 M 
1 | tower2      | BertModel | 109 M 
2 | dropout     | Dropout   | 0     
3 | final_layer | Linear    | 98 K  


1

In [ ]:
trainer.test(model)

## Colocando dados no formato do trec_eval

In [ ]:
qids = []
pdis = []
sims = []
ranks = []
runid = 'runid1'
results_path = proj_dir + '/Data/Checkpoints_dense_DOIS_MOD/' + 'results.test.txt' 

with open(results_path, 'w') as text_file:
  for batch in val_dataloader:
      qid,pids,o_query,passages,o_query_tok,o_query_mask,passages_tensors = batch
      o_query_tok = o_query_tok.to(device)
      o_query_mask = o_query_mask.to(device)
      passages_tensors = [(tuple_pasasge[0].to(device), tuple_pasasge[1].to(device)) for tuple_pasasge in  passages_tensors]
      batch_gpu = qid,pids,o_query,passages,o_query_tok,o_query_mask,passages_tensors

      
      _, batch_pids, batch_qids , batch_similarities = model.infer(batch_gpu)
      for pids, qids , similarities in zip(batch_pids, batch_qids , batch_similarities):
        ranks = [i+1 for i in range (0,len(qids))]
        for p, q, s, r in zip(pids, qids , similarities, ranks):
          text_file.write(q + " " + "Q0" + " " + p + " " + str(r) + " " + str(s) + " " + runid + "\n") 
        break

RuntimeError: ignored

In [ ]:
# compression_opts = dict(method='zip', archive_name='enpt_t5_results.csv') 
# model.df_test.to_csv('enpt_t5_results.zip', index=False,compression=compression_opts)  

In [ ]:
[i+1 for i in range (0,5)]

[1, 2, 3, 4, 5]

## Conclusão

# Fim do Notebook

In [ ]:
%%capture
!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

In [ ]:
import numpy as np
a = np.random.rand(1, 64).astype('float32')
b = np.random.rand(50, 64).astype('float32')

In [ ]:
a

array([[0.03288486, 0.99930036, 0.0917538 , 0.9760556 , 0.17281024,
        0.20732298, 0.75187033, 0.60691273, 0.94161123, 0.43548256,
        0.0511204 , 0.8335275 , 0.45162374, 0.18560098, 0.5455756 ,
        0.9506205 , 0.6063954 , 0.5880484 , 0.10234836, 0.4813956 ,
        0.9589875 , 0.11445889, 0.11558983, 0.92979205, 0.84923166,
        0.85942894, 0.8659117 , 0.6770767 , 0.21826208, 0.216732  ,
        0.87385005, 0.80729824, 0.79571456, 0.953996  , 0.22395045,
        0.36522022, 0.92701423, 0.30594   , 0.00745852, 0.12038016,
        0.6518728 , 0.723216  , 0.6868079 , 0.30146527, 0.86717075,
        0.91433847, 0.0791575 , 0.58189446, 0.8286191 , 0.86756873,
        0.8740032 , 0.7332599 , 0.66985065, 0.9416499 , 0.37234637,
        0.6477211 , 0.53455114, 0.07596271, 0.7203342 , 0.32556063,
        0.84339446, 0.41703832, 0.7262496 , 0.05590044]], dtype=float32)

In [ ]:
import faiss        
d =64          # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(b)                  # add vectors to the index
print(index.ntotal)

True
50


In [ ]:
D, I = index.search(a, 50)     

In [ ]:
D

array([[ 9.470676 ,  9.647051 , 10.046372 , 10.057165 , 10.457943 ,
        10.637009 , 10.666663 , 10.71827  , 10.816111 , 11.009465 ,
        11.139634 , 11.145186 , 11.150333 , 11.3476   , 11.377235 ,
        11.573856 , 11.642075 , 11.69183  , 11.879677 , 11.883938 ,
        11.918463 , 11.957674 , 11.963694 , 12.129216 , 12.167028 ,
        12.176181 , 12.1870575, 12.202099 , 12.247135 , 12.344605 ,
        12.350199 , 12.565917 , 12.717619 , 12.767759 , 12.784656 ,
        12.803101 , 12.906161 , 12.956519 , 13.184155 , 13.284876 ,
        13.344904 , 13.390293 , 13.4876   , 13.682433 , 13.693873 ,
        13.990819 , 14.580568 , 15.360914 , 15.627806 , 16.341415 ]],
      dtype=float32)

In [ ]:
I

array([[42, 35,  8, 36,  7, 49, 24, 17, 19,  9, 47, 39, 12, 29,  2, 25,
        27,  1,  6, 26, 46, 43, 14, 11, 34,  3, 33,  0, 15,  5, 40, 22,
         4, 31, 38, 37, 32, 45, 13, 21, 18, 28, 10, 48, 20, 16, 41, 44,
        30, 23]])